In [ ]:
import os
import configparser

from sqlalchemy import create_engine

from tabulate import tabulate

import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
plt.style.use('seaborn-poster')

In [ ]:
config = configparser.ConfigParser()
config.read(os.path.join(os.path.expanduser('~'), '.config', 'me.ini'))

dburl = config['db']['url']
engine = create_engine(dburl)

In [ ]:
# read database data

df = pd.read_sql('''
    select
        cur.*,
        mkt.price_btc,
        mkt.price_eur,
        mkt.percent_change_1h as change
    from cryptomarket mkt right join cryptocurrencies cur on mkt.read_date = cur.read_date and mkt.symbol = cur.coin
''', engine)

df.loc[df.coin =='EUR', 'price_eur'] = 1
df.loc[df.coin =='EUR', 'price_btc'] = 1 / df[df.coin == 'BTC'].price_eur.max()
df['btc_value'] = df.balance * df.price_btc
df['eur_value'] = df.balance * df.price_eur

In [ ]:
# Compare price in Euro and in BTC

fig = plt.figure()
ax1 = fig.add_subplot(111)
ax2 = ax1.twinx()

df.pivot_table(index='read_date', values=['btc_value'], aggfunc=sum).plot(ax=ax1, color='r')
df.pivot_table(index='read_date', values=['eur_value'], aggfunc=sum).plot(ax=ax2)

In [ ]:
# Compare price in Euro and in BTC for Litecoin and Ethereum

fig = plt.figure()
ax1 = fig.add_subplot(111)
ax2 = ax1.twinx()

df[df.coin.isin(['LTC', 'ETH'])].pivot_table(index='read_date', columns='coin', values=['btc_value'], aggfunc=sum).plot(ax=ax1, color='r')
df[df.coin.isin(['LTC', 'ETH'])].pivot_table(index='read_date', columns='coin', values=['eur_value'], aggfunc=sum).plot(ax=ax2, color='b')

In [ ]:
# BTC value for all other coins.
df[~df.coin.isin(['BTC', 'LTC', 'ETH'])].pivot_table(index='read_date', columns='coin', values=['btc_value'], aggfunc=sum).plot()

In [ ]:
# Euro value for all other coins.
df[~df.coin.isin(['BTC', 'LTC', 'ETH'])].pivot_table(index='read_date', columns='coin', values=['eur_value'], aggfunc=sum).plot()

In [ ]:
# current values, as of last reading
portfolio = df[df.read_date == df.read_date.max()]
portfolio.sort_values(by='source')

In [ ]:
# Table with Euro and BTC values
portfolio.groupby(['name'])['btc_value', 'eur_value', 'change'].sum().reset_index().sort_values(by='name', ascending=True)

In [ ]:
# Total values for the portfolio

btc = portfolio.btc_value.sum()
eur = portfolio.eur_value.sum()
btc_eur = df[(df.read_date == df.read_date.max()) & (df.name == 'Bitcoin')].price_eur.max()

print(f'Total is {eur:.2f} € or {btc} btc. This would be {btc_eur * btc:.2f} € if it was all BTC.')

In [ ]:
# Euro percentage in each coin.
portfolio.groupby('name').eur_value.sum().plot.pie(autopct='%.2f')

In [ ]:
# Euro percentage in each exchange or wallet
portfolio.groupby('source').eur_value.sum().plot.pie(autopct='%.2f')